In [1]:
#packages
import json
import tqdm
import os
import numpy as np
import sys
import glob
import pandas as pd
import datetime as dt
import re
import math
from collections import defaultdict
import operator

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import preprocessing_Origins, general_cleaning, verification_based_on_initial_record,\
ZoneVariable, HenDailyVariable_Origins, boxplot_distribution_entropy, heatmap_duration_perzone_perhen, is_day
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
#initialise parameter
path_initial_data = config.path_initial_data
path_extracted_data_daily_check = config.path_extracted_data_daily_check
id_run = config.id_run
dico_rc_sys = config.dico_rc_sys
path_FocalBird = config.path_FocalBird

focal_name = config.focal_name
path_Days = config.path_Days

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
#voltage thresholds
#Threshold for minimarker and wintergarden battery change
thresh_dev = 3100
#Threshold for tag battery change
thresh_tag = 2930

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download data

In [5]:
#for the logs and device updates
over_last_Xdays = 6

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### logs with their associated HenID

In [6]:
p = glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\TagUpdates\log*'))
#mtime = os.stat(os.path.join('.', p[0])).st_mtime
#download_time = dt.datetime.fromtimestamp(mtime).strftime('%Y-%m-%d-%H:%M')
#download_time = dt.datetime.strptime(download_time, '%Y-%m-%d-%H:%M')
#print('The file were downloaded at: ', download_time)
    
#print('There is %d log files:\n  %s'%(len(p),'  \n  '.join(p)))
df = preprocessing_Origins(p, config, save=False, dodevice=False)
print(df.shape)
display(df.head(3))
display(df.tail(3))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:1: DeprecationWarning: invalid escape sequence \T
<>:1: DeprecationWarning: invalid escape sequence \T
<ipython-input-6-49996d438290>:1: DeprecationWarning: invalid escape sequence \T
  p = glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\TagUpdates\log*'))


From the focalBirdinfo, you have 160 ative tags


C:\Users\camil\Desktop\animals_code\PhD\UTILS.py:402: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_ = df.groupby(['HenID'])['system','PenID','TagID'].agg(lambda x: set(x)).reset_index()


(5149587, 47)


,Timestamp,TagSerialNumber,TagID,Zone,systemgantnerid,useless_zone,signalstrength,zone2,signalstzone2,zone3,...,backpackUniqueID,EPI GENETIC TIMESTAMP,HA 23-11-2020,comportement,weight 23-11-2020,HA 04-01-2021,weight 04-01-2021,HA 01-02-21,weight 01-02-21,weight 12-04-21
0,2020-09-29 09:07:00,02001E13,162,Tier 2 (mini 12),2,8,10,2,8,1,...,12wp,10-date:4-2021,NaN,NaN,"1708,4",the student wrote the weight in kg only one di...,"1800, +70 -30",NaN,"1820,4",1913
120,2020-09-29 09:08:12,02001E6A,140,Tier 2 (mini 11),2,7,10,2,8,1,...,11wp,NaN,NaN,NaN,"1788,9",NaN,"1905,1",NaN,"1905,6","2046,7"
1,2020-09-29 09:08:26,02001E13,162,Tier 2 + untere Stange,2,2,11,8,10,3,...,12wp,10-date:4-2021,NaN,NaN,"1708,4",the student wrote the weight in kg only one di...,"1800, +70 -30",NaN,"1820,4",1913


,Timestamp,TagSerialNumber,TagID,Zone,systemgantnerid,useless_zone,signalstrength,zone2,signalstzone2,zone3,...,backpackUniqueID,EPI GENETIC TIMESTAMP,HA 23-11-2020,comportement,weight 23-11-2020,HA 04-01-2021,weight 04-01-2021,HA 01-02-21,weight 01-02-21,weight 12-04-21
2136944,2021-04-26 09:22:10,02002421,198,Tier 2 + untere Stange,2,2,12,8,10,1,...,12ps,NaN,NaN,NaN,"1757,9",the student wrote the weight in kg only one di...,"1800, +70 -30",batterie,"1808,9","1969,6"
2133908,2021-04-26 09:22:15,02001E70,177,Tier 2 (mini 12),2,8,17,2,8,0,...,12ss,NaN,NaN,NaN,1580,the student wrote the weight in kg only one di...,"1600, +70 -30",NaN,"1640,1","1685,1"
2136945,2021-04-26 09:22:19,02002421,198,Tier 2 (mini 12),2,8,12,2,8,0,...,12ps,NaN,NaN,NaN,"1757,9",the student wrote the weight in kg only one di...,"1800, +70 -30",batterie,"1808,9","1969,6"


In [7]:
df_FB = pd.read_csv(path_FocalBird, sep=';', parse_dates=['StartDate','EndDate'], dayfirst=True, encoding='latin') #verified dates:correct
#fill end date to today+1 for the birds which we dont know when is there end date (+1: so that today is taken into account)
df_FB['EndDate'].fillna(config.date_max+dt.timedelta(days=1), inplace=True)
df_FB['TagID'] = df_FB['TagID'].astype(str)
df_FB['PenID'] = df_FB['PenID'].map(int).map(str)
#exclude rows were tags were not functionning correctly for some reason 
df_FB = df_FB[df_FB['ShouldBeExcluded']!='yes']
#define a list with the active tags of today/last date
li_active_tags = list(df_FB[df_FB['EndDate']>=config.date_max]['TagID'].unique())
#Counter(li_active_tags)
print('From the focalBirdinfo, you have %d ative tags'%len(li_active_tags))

From the focalBirdinfo, you have 160 ative tags


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### active tags

In [8]:
df_tag = pd.read_csv(path_FocalBird, sep=';', parse_dates=['StartDate','EndDate'], dayfirst=True, encoding='latin')
df_tag['EndDate'].fillna(dt.date.today(), inplace=True)
print(df_tag.shape)
df_tag = df_tag[df_tag['ShouldBeExcluded']!='yes']
df_tag['TagID'] = df_tag['TagID'].astype(str)
print(df_tag.shape)
df_tag.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(265, 29)
(251, 29)


,HenID,R-Pen,InitialStartDate,ShouldBeExcluded,StartDate,EndDate,FocalLegringID,Expected % of wrong class,Expected chance of wrong treatment,comments,...,TagID,EPI GENETIC TIMESTAMP,HA 23-11-2020,comportement,weight 23-11-2020,HA 04-01-2021,weight 04-01-2021,HA 01-02-21,weight 01-02-21,weight 12-04-21
0,1,1.0,10.06.2020,NaN,2020-09-28,2021-02-01 00:00:00,3,NaN,0,NaN,...,105,9-date:2-2020,NaN,NaN,"1696,5",NaN,"1787,8",NaN,"1800,9",NaN
1,2,1.0,10.06.2020,NaN,2020-09-28,2021-04-26,7,NaN,0,NaN,...,18,NaN,NaN,NaN,"1734,9",NaN,"1691,7",NaN,"1777,8","1782,2"
2,3,1.0,10.06.2020,NaN,2020-09-28,2021-04-26,8,NaN,0,NaN,...,114,NaN,NaN,NaN,"1499,4",NaN,"1714,7",NaN,"1838,3","1887,4"


In [9]:
li_active_tags = list(set(df_tag[df_tag['EndDate']>=dt.date.today()]['TagID'].unique()))
#Counter(li_active_tags)
print('From your focalbird document you should have %d ative tags'%len(li_active_tags))

From your focalbird document you should have 160 ative tags


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### device

In [10]:
#select last x logs
last_x_logs = math.ceil(over_last_Xdays/2) #arondi en haut
li_logs = []
for path_system in glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\DeviceUpdates')):
    li_ = glob.glob(os.path.join(path_system, 'log*'))
    li_.sort(key=lambda x: os.path.getmtime(x), reverse=False)
    li_logs.extend(li_[-last_x_logs:]) 
#next version
#li_newFirmwarecol = ['Timestamp','sort','code','sender','Temperature','Battery Voltage','MovementCounter','LastLFSeen',
#                     'LFCounter','LFRSSISum','Zone', 'Subzone','LFRSSI','Time','LastInfo','LastLocation','LastAction']
li_newFirmwarecol = ['Timestamp','sort','code','sender','Temperature','Battery Voltage','MovementCounter','LastLFSeen',
                     'LFCounter','LFRSSISum','Zone', 'Subzone','Time','LastInfo','LastLocation','LastAction']
#lastReg = LastAction
li_df = []
for path_log in tqdm.tqdm(li_logs):
    #confused month and day
    df_device = pd.read_csv(os.path.join(path_log), sep=';', names=li_newFirmwarecol, parse_dates=['LastAction','Timestamp'],
                           dayfirst=True)
    df_device['data_path'] = path_log
    df_device['system'] = 'pens:'+path_log.split('Barn 4 Pen ')[1].split('\\')[0]
    df_device['logID'] = path_log.split('\\')[-1].split('.')[0]
    print(df_device.shape)
    print('-----------------', path_log)
    #display(df_device.head(3))
    #display(df_device.tail(3))
    li_df.append(df_device)
df_device = pd.concat(li_df)
df_device = df_device[df_device['Timestamp']>=dt.datetime(2020,9,29)]
df['date'] = df['Timestamp'].map(lambda x: dt.datetime.date(x))
df_device['sender'].fillna(' ', inplace=True)
#add the info on if the device needs batterie or not
df_device['NeedsBatteries'] = df_device.apply(lambda x: ('mini' in x['sender']) | (x['sender']=='Wintergarten') | \
                                              (x['sort']=='Tag'), axis=1)
#remove old tags
print(df_device.shape)
df_device = df_device[~((df_device['sort']=='Tag')&(~df_device['sender'].isin(li_active_tags)))]
print(df_device['logID'].unique())
print(df_device.shape)
df_device.tail(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:4: DeprecationWarning: invalid escape sequence \D
<>:4: DeprecationWarning: invalid escape sequence \D
<ipython-input-10-f27d1d550a49>:4: DeprecationWarning: invalid escape sequence \D
  for path_system in glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\DeviceUpdates')):
 11%|█████████▎                                                                          | 1/9 [00:28<03:47, 28.48s/it]

(214900, 19)
----------------- G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\GantnerSystem\_dailycheckingSystem\Barn 4 Pen 10 - 12\DeviceUpdates\log_00000015.csv


 22%|██████████████████▋                                                                 | 2/9 [00:56<03:18, 28.41s/it]

(214900, 19)
----------------- G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\GantnerSystem\_dailycheckingSystem\Barn 4 Pen 10 - 12\DeviceUpdates\log_00000016.csv


 33%|████████████████████████████                                                        | 3/9 [01:16<02:34, 25.80s/it]

...

(223380, 19)
----------------- G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\GantnerSystem\_dailycheckingSystem\Barn 4 Pen 8 - 9\DeviceUpdates\log_00000011.csv


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [03:30<00:00, 23.37s/it]

(199500, 19)
----------------- G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\GantnerSystem\_dailycheckingSystem\Barn 4 Pen 8 - 9\DeviceUpdates\log_00000012.csv


(1693384, 20)
['log_00000015' 'log_00000016' 'log_00000017' 'log_00000018'
 'log_00000010' 'log_00000011' 'log_00000012']
(1623657, 20)


,Timestamp,sort,code,sender,Temperature,Battery Voltage,MovementCounter,LastLFSeen,LFCounter,LFRSSISum,Zone,Subzone,Time,LastInfo,LastLocation,LastAction,data_path,system,logID,NeedsBatteries
199497,2021-04-26 09:11:03,Tag,02002428,195,27.0,2976.0,46.0,0.0,404.0,5963.0,3.0,2.0,97.0,26.04.2021 09:04:29,26.04.2021 09:02:49,2021-04-26 09:10:40,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,pens:8 - 9,log_00000012,True
199498,2021-04-26 09:11:03,Tag,02002332,196,28.0,2982.0,22.0,1.0,408.0,7269.0,3.0,4.0,255.0,26.04.2021 09:04:20,26.04.2021 09:10:46,2021-04-26 09:10:46,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,pens:8 - 9,log_00000012,True
199499,2021-04-26 09:11:03,Tag,02002333,197,29.0,3005.0,73.0,3.0,431.0,4788.0,3.0,2.0,255.0,26.04.2021 09:08:27,26.04.2021 09:10:00,2021-04-26 09:10:00,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,pens:8 - 9,log_00000012,True


In [11]:
#df_device.groupby(['system','logID'])['NeedsBatteries'].count().reset_index()

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
#df_device[df_device['logID']=='log_00000010']

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
#an error might be cause if the alst row was not complet when the file was downloaded

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
#df_device[df_device['sort']=='Marker'][['code','sender','system']].drop_duplicates() #all actives at least once

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
#df_device[(df_device['sort']=='Tag')&(df_device['Battery Voltage']<0)]

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Use only the last few days

In [16]:
download_time = max(df_device['Timestamp'].tolist())

#remove the days we dont want
df_device['to_analyse'] = df_device['Timestamp'].map(lambda x: x>=(download_time-dt.timedelta(days=over_last_Xdays)))
df_device = df_device[df_device['to_analyse']]

path_daily_ver = os.path.join(path_extracted_data_daily_check,str(download_time).split(' ')[0])
#create a director if not existing
if not os.path.exists(path_daily_ver):
    os.makedirs(path_daily_ver)

#keep only the last over_last_Xdays days in the logs
df['to_analyse'] = df['Timestamp'].map(lambda x: x>=(download_time-dt.timedelta(days=over_last_Xdays)))
df = df[df['to_analyse']]
df[df['to_analyse']]['date'].unique()    

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([datetime.date(2021, 4, 20), datetime.date(2021, 4, 21),
       datetime.date(2021, 4, 22), datetime.date(2021, 4, 23),
       datetime.date(2021, 4, 24), datetime.date(2021, 4, 25),
       datetime.date(2021, 4, 26)], dtype=object)

In [17]:
download_time

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Timestamp('2021-04-26 09:19:55')

# Tags last records

In [18]:
#tags last records and active tags with no records
df_ = df[df['TagID'].isin(li_active_tags)].groupby(['TagID','PenID'])['Timestamp'].agg(lambda x: list(x)).reset_index()
df_['last_record_timestamp'] = df_['Timestamp'].map(lambda x: max(x))
df_['nbr_h_since_lastrecord'] = df_['last_record_timestamp'].map(lambda x: int((download_time-x).total_seconds()/60/60))

#add active tags that had no transition at all
li_no_transition = [i for i in li_active_tags if i not in df['TagID'].unique()]
df_no_transition = pd.DataFrame([{'TagID':tag_, 'nbr_h_since_lastrecord':9999} for tag_ in li_no_transition])
print(df_no_transition.shape, df_.shape)
df_ = pd.concat([df_, df_no_transition])
print(df_.shape)
#remove the older row of tags taht appear two times (i.e. if they changed pens for example)
df_.sort_values(['nbr_h_since_lastrecord'], ascending=False, inplace=True)
df_.drop_duplicates(subset=['TagID'], keep='last', inplace=True)
df_[['TagID','PenID','last_record_timestamp','nbr_h_since_lastrecord']].to_csv(os.path.join(path_daily_ver, 
                                                                           id_run+str(download_time).split(' ')[0]+'h_TagsLastRecords.csv'), 
                                                              sep=';', index=False)

h = 10
print('------------- Tags with no records since at least %d hours: -------------'%h)
print('There is %d such tags, we will print a maximum of 50 line (starting form the worst ones)'%df_[df_['nbr_h_since_lastrecord']>h].shape[0])
li_tag_issues = df_[df_['nbr_h_since_lastrecord']>h][['TagID','PenID','last_record_timestamp','nbr_h_since_lastrecord']]['TagID'].tolist()
print('There is %d tags with at least one issue'%len(set(li_tag_issues)))
df_tag_summary = df_.copy()
df_[df_['nbr_h_since_lastrecord']>h][['TagID','PenID','last_record_timestamp','nbr_h_since_lastrecord']].head(50)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, 0) (160, 5)
(160, 5)
------------- Tags with no records since at least 10 hours: -------------
There is 0 such tags, we will print a maximum of 50 line (starting form the worst ones)
There is 0 tags with at least one issue


,TagID,PenID,last_record_timestamp,nbr_h_since_lastrecord


In [19]:
#df[df['TagID']=='19'] #10.04 morning: verify in video
#df_device[df_device['sender']=='29']

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Reader last update

#understand how the device logs are
df_test = df_device.groupby('code')['system'].agg(lambda x: len(set(x))).reset_index()
df_test[df_test['system']>1] #--> no code in two system!!
#importantly:
df_device[df_device['sort']!='Tag'].groupby(['sender'])['system','code','sort'].agg(lambda x: set(x)).reset_index()

In [20]:
for reader_code in dico_rc_sys.keys():
    print("The last read out from the reader of %s arrived at: %s"%(dico_rc_sys[reader_code],
                                                    str(max(df_device[df_device['code']==reader_code]['LastAction'].tolist()))))

The last read out from the reader of Reader Pen 3-5 arrived at: 2021-04-26 09:11:10
The last read out from the reader of Reader Pen 8-9 arrived at: 2021-04-26 09:10:59
The last read out from the reader of Reader Pen 10-12 arrived at: 2021-04-26 09:19:52


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#cleaning step: remove days for specific hens when the tag was not sending output regularly anymore
df_test = df_device[df_device['sort']=='Tag'].copy()
df_test['date'] = df_test['Timestamp'].map(lambda x: dt.datetime.date(x))
df_test = df_test.groupby(['sender','date','hour'])['LastAction'].agg(lambda x: list(x)).reset_index()
df_test.rename(columns={'sender':'TagID'}, inplace=True)
df_test['nbr_last_action']
#last action biggest gap
print(df_test.shape)
df_test.head(3)

## LF Counter

In [21]:
df_LFCoutner = df_device[df_device['sort']=='Tag'].copy()
df_LFCoutner['date'] = df_LFCoutner['Timestamp'].map(lambda x: dt.datetime.date(x))
#df_LFCoutner['is_day'] = df_LFCoutner['Timestamp'].map(lambda x: is_day(x, config.dico_night_hour))
df_LFCoutner = df_LFCoutner.groupby(['sender','date'])['LFCounter'].agg(lambda x: sorted(list(x))).reset_index()
df_LFCoutner['LFCounter_nbr_equal0'] = df_LFCoutner['LFCounter'].map(lambda x: sum([i==0 for i in x]))
df_LFCoutner['LFCounter_atleastone0'] = df_LFCoutner['LFCounter_nbr_equal0'].map(lambda x: x>0)
print(df_LFCoutner.shape)
df_ = df_LFCoutner[df_LFCoutner['LFCounter_atleastone0']].groupby(['sender'])[['date',
                                                                'LFCounter_nbr_equal0']].agg(lambda x: list(x)).reset_index()
df_.to_csv(os.path.join(path_daily_ver, id_run+str(download_time).split(' ')[0]+'_LFCounterEqual0.csv'),sep=';')
df_

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1120, 5)


,sender,date,LFCounter_nbr_equal0
0,102,"[2021-04-20, 2021-04-22, 2021-04-23, 2021-04-2...","[1, 7, 1, 2, 1]"
1,104,"[2021-04-21, 2021-04-23, 2021-04-25, 2021-04-26]","[1, 2, 2, 1]"
2,105,"[2021-04-20, 2021-04-21, 2021-04-22, 2021-04-2...","[5, 1, 1, 2, 6]"
3,106,"[2021-04-21, 2021-04-22, 2021-04-23, 2021-04-2...","[2, 2, 2, 4, 1, 1]"
4,108,"[2021-04-21, 2021-04-22, 2021-04-24, 2021-04-25]","[2, 1, 1, 2]"
...,...,...,...
132,92,"[2021-04-21, 2021-04-22, 2021-04-24]","[1, 2, 1]"
133,94,"[2021-04-22, 2021-04-24, 2021-04-25]","[1, 1, 2]"
134,95,"[2021-04-20, 2021-04-22, 2021-04-23, 2021-04-2...","[1, 1, 1, 1, 2, 1]"
135,98,[2021-04-26],[1]


# Device Last update

In [22]:
#TODO
#New data!!!: plt.hist(df_device['MovementCounter'].dropna(),bins='auto');
#plt.hist(df_device['Temperature'].dropna(),bins=30);

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## biggest gap over the entire day

In [23]:
df_lastactionGAP = df_device[df_device['sort']=='Tag'].copy()
df_lastactionGAP['date'] = df_lastactionGAP['Timestamp'].map(lambda x: dt.datetime.date(x))
#df_lastactionGAP['is_day'] = df_lastactionGAP['Timestamp'].map(lambda x: is_day(x, config.dico_night_hour))
#df_lastactionGAP = df_lastactionGAP[df_lastactionGAP['is_day']]
df_lastactionGAP = df_lastactionGAP.groupby(['sender','date'])['LastAction'].agg(lambda x: sorted(list(x))).reset_index()
#last action biggest gap of the day
df_lastactionGAP['li_gap_of_the_day_tsstarted'] = df_lastactionGAP['LastAction'].map(lambda x: {x[i]:(x[i+1]-x[i]).total_seconds() for i in range(0,len(x)-1)})
df_lastactionGAP['biggest_gap_of_day_mn'] = df_lastactionGAP['li_gap_of_the_day_tsstarted'].map(lambda x: round(max(x.items(), 
                                                                                         key=operator.itemgetter(1))[1]/60,0))
df_lastactionGAP['start_biggest_gap_of_day'] = df_lastactionGAP['li_gap_of_the_day_tsstarted'].map(lambda x: max(x.items(), 
                                                                                               key=operator.itemgetter(1))[0])
df_lastactionGAP.to_csv(os.path.join(path_daily_ver, id_run+str(download_time).split(' ')[0]+'_LastactionGAP.csv'),sep=';')
print(df_lastactionGAP.shape)
df_lastactionGAP[df_lastactionGAP['biggest_gap_of_day_mn']>12]

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1120, 6)


,sender,date,LastAction,li_gap_of_the_day_tsstarted,biggest_gap_of_day_mn,start_biggest_gap_of_day
1099,95,2021-04-20,"[2021-04-20 09:20:34, 2021-04-20 09:30:38, 202...","{2021-04-20 09:20:34: 604.0, 2021-04-20 09:30:...",183.0,2021-04-20 10:06:40


### very last update

In [24]:
#sort by date in order to take the last voltage using a simple list
df_ = df_device.groupby(['code','sender','system','sort','NeedsBatteries'])['LastAction'].agg(lambda x: list(x)).reset_index()
df_['last_LastAction'] = df_['LastAction'].map(lambda x: max(x))
df_['nbr_mn_since_LastUpdate'] = df_['last_LastAction'].map(lambda x: (download_time-x).total_seconds()/60)

#add tags with no update:
li_no_transition = [i for i in li_active_tags if i not in df_device['sender'].unique()]
df_no_transition = pd.DataFrame([{'sender':tag_, 'nbr_mn_since_LastUpdate':9999} for tag_ in li_no_transition])
print(df_no_transition.shape, df_.shape)
df_ = pd.concat([df_, df_no_transition])
print(df_.shape)
print('There is %d device with 0 updates'%df_no_transition.shape[0])
df_.sort_values(['nbr_mn_since_LastUpdate'], ascending=False, inplace=True)
#remove the older sender that appear two times (i.e. if they changed pens for example)
df_.drop_duplicates(subset=['system','sender'], keep='last', inplace=True)
df_[['code','sender','system','sort','last_LastAction','nbr_mn_since_LastUpdate']].to_csv(os.path.join(path_daily_ver, 
                                            id_run+str(download_time).split(' ')[0]+'h_DeviceLastUpdate.csv'), sep=';', index=False)


mn = 12
li_tag_issues.extend(df_[df_['nbr_mn_since_LastUpdate']>mn]['sender'])
print('There is %d tags with at least one issue'%len(set(li_tag_issues)))

print('------------- Device with no update since at least %d minutes -------------'%mn)
df_tag_summary = pd.merge(df_tag_summary, df_[df_['sort']=='Tag'], left_on='TagID', right_on='sender', how='outer')
#display(df_tag_summary.head(3))
display(df_[df_['nbr_mn_since_LastUpdate']>mn][['code','sender','system','sort','last_LastAction','nbr_mn_since_LastUpdate']])

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, 0) (187, 8)
(187, 8)
There is 0 device with 0 updates
There is 1 tags with at least one issue
------------- Device with no update since at least 12 minutes -------------


,code,sender,system,sort,last_LastAction,nbr_mn_since_LastUpdate
63,02001571,71,pens:10 - 12,Tag,2021-04-26 03:01:01,378.9


In [25]:
download_time

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Timestamp('2021-04-26 09:19:55')

# Critical Voltage last D days

In [26]:
D = 7 #assuming I do this one a week, it make sense to plot it once a week
#D=2 was not enough as we would miss some tags that were at 2600 (e.g. 82)
df_device['is_last_'+str(D)+'days'] = df_device['Timestamp'].map(lambda x: x>=(download_time-dt.timedelta(days=D)))
df_ = df_device[df_device['is_last_'+str(D)+'days']].groupby(['code','sender','system','sort',
                                                        'NeedsBatteries'])['Battery Voltage'].agg(lambda x: list(x)).reset_index()
df_['min_Battery_Voltage_last_'+str(D)+'days'] = df_['Battery Voltage'].map(lambda x: min(x))
df_['avg_Battery_Voltage_last_'+str(D)+'days'] = df_['Battery Voltage'].map(lambda x: np.nanmean(x))
df_.sort_values(['min_Battery_Voltage_last_'+str(D)+'days'], ascending=False, inplace=True)
# check whether at least one of the last entries was below the critical threshold 
df_['CRITICAL'] = df_.apply(lambda x:sum([i<thresh_dev for i in x['Battery Voltage']])>=1 if x['sort']!='Tag' \
                            else sum([i<thresh_tag for i in x['Battery Voltage']])>=1, axis=1)
df_.to_csv(os.path.join(path_daily_ver,id_run+str(download_time).split(' ')[0]+'h_Battery VoltageLast'+str(D)+'days.csv'), sep=';', index=False)
print('------------- Critical Battery Voltage of device/tags (under threshold at least once during the last %d days -------------'%D)
li_tag_issues.extend(df_[(df_['CRITICAL'])&(df_['sort']=='Tag')]['sender'])
print(df_tag_summary.shape)
df_tag_summary = df_tag_summary.drop(['sender'], axis=1)
df_tag_summary = pd.merge(df_tag_summary, df_[df_['sort']=='Tag'][['CRITICAL','min_Battery_Voltage_last_'+str(D)+'days',
                                                                   'avg_Battery_Voltage_last_'+str(D)+'days','sender']], 
                          left_on='TagID', right_on='sender', how='outer')
df_tag_summary.drop(['LastAction','Timestamp','NeedsBatteries'], axis=1).to_csv(os.path.join(path_daily_ver,id_run+str(download_time).split(' ')[0]+'_TagSummary.csv'), sep=';', index=False)
print(df_tag_summary.shape)
#display(df_tag_summary.head(3))
print('There is %d tags with at least one issue:'%len(set(li_tag_issues)))
df_[(df_['CRITICAL'])&(df_['NeedsBatteries'])]

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-26-207c442b845e>:7: RuntimeWarning: Mean of empty slice
  df_['avg_Battery_Voltage_last_'+str(D)+'days'] = df_['Battery Voltage'].map(lambda x: np.nanmean(x))


------------- Critical Battery Voltage of device/tags (under threshold at least once during the last 7 days -------------
(160, 13)
(160, 16)
There is 25 tags with at least one issue:


,code,sender,system,sort,NeedsBatteries,Battery Voltage,min_Battery_Voltage_last_7days,avg_Battery_Voltage_last_7days,CRITICAL
19,0200141D,48,pens:3 - 5,Tag,True,"[3022.0, 3020.0, 3018.0, 3023.0, 3023.0, 3021....",2928.0,3022.120370,True
34,02001529,29,pens:3 - 5,Tag,True,"[2987.0, 2984.0, 2987.0, 2988.0, 2988.0, 2975....",2926.0,2987.465278,True
93,02001E14,128,pens:10 - 12,Tag,True,"[3020.0, 3021.0, 3022.0, 3020.0, 3022.0, 3024....",2926.0,3014.552601,True
59,02001564,64,pens:3 - 5,Tag,True,"[2980.0, 2979.0, 2980.0, 2976.0, 2982.0, 2980....",2923.0,2977.398148,True
77,020015C5,108,pens:8 - 9,Tag,True,"[3009.0, 3005.0, 2999.0, 3002.0, 3002.0, 3002....",2922.0,2996.960648,True
25,02001424,24,pens:3 - 5,Tag,True,"[3030.0, 3033.0, 3037.0, 3034.0, 3032.0, 3036....",2921.0,3032.013889,True
23,02001422,14,pens:3 - 5,Tag,True,"[2991.0, 2996.0, 2993.0, 2993.0, 2995.0, 2994....",2920.0,2990.202546,True
30,0200151D,41,pens:3 - 5,Tag,True,"[2996.0, 2997.0, 2999.0, 3000.0, 2999.0, 3001....",2920.0,2997.738426,True
96,02001E17,145,pens:10 - 12,Tag,True,"[2990.0, 2994.0, 2991.0, 2991.0, 2989.0, 2915....",2915.0,2993.741040,True
66,02001579,79,pens:8 - 9,Tag,True,"[2969.0, 2973.0, 2976.0, 2972.0, 2973.0, 2972....",2907.0,2979.947917,True


#plot

df_device = df_device.sort_values(['LastAction'])
for tag in li_active_tags:
    df_plt = df_device[df_device['sender']==str(tag)][['voltage','Timestamp']]
    plt.plot(df_plt['Timestamp'].tolist(), df_plt['voltage'].tolist());
    plt.title(tag)
    plt.show()

# Plot Tags

#plot the batterie level, the tagupdates and the number of transition for each tags, one plot per day
df_device['date'] = df_device['Timestamp'].map(lambda x: dt.datetime.date(x))
df_device = df_device.sort_values(['LastAction'])
#us the log files to crate a df_tr file with one row per hen per hour, and the number of transition in a column
df['hour'] = df['Timestamp'].map(lambda x: x.hour if pd.isnull(x)!=np.nan else np.nan)
df_tr = df.groupby(['TagID','date','hour'])['Timestamp'].count().reset_index()
df_tr.rename(columns={'Timestamp':'nbr_transition_last1h'}, inplace=True)
df_tr['date_hour'] = df_tr.apply(lambda x: dt.datetime(x['date'].year, x['date'].month, x['date'].day, x['hour'],0,0), axis=1)

#for tag in tqdm.tqdm(set(li_tag_issues)):
for tag in tqdm.tqdm(li_active_tags):
    try:
        plt.figure(figsize=(18,4))
        ax = plt.subplot(111) 

        #Battery Voltage
        df_plt = df_device[df_device['sender']==tag].copy()
        SystemID = df_plt['system'].iloc[0]
        ax.plot(df_plt['Timestamp'].tolist(), df_plt['Battery Voltage'].tolist(), color='blue');

        #number of transition in the last hours    
        ax2=ax.twinx()
        df_tr_ = df_tr[df_tr['TagID']==tag].copy()
        ax2.plot(df_tr_['date_hour'].tolist(), df_tr_['nbr_transition_last1h'].tolist(), color='green');

        #number of device update in the last hour
        df_plt['hour'] = df_plt['Timestamp'].map(lambda x: x.hour if pd.isnull(x)!=np.nan else np.nan)
        df_up = df_plt.groupby(['date','hour'])['LastAction'].agg(lambda x: list(x)).reset_index()
        df_up['date_hour'] = df_up.apply(lambda x: dt.datetime(x['date'].year, x['date'].month, x['date'].day, x['hour'],0,0), axis=1)
        df_up['nbr_update_last1h'] = df_up.apply(lambda x: sum([i>=(x['date_hour']-dt.timedelta(hours=1)) for i in x['LastAction']]), 
                                                 axis=1)
        ax2.plot(df_up['date_hour'].tolist(), df_up['nbr_update_last1h'].tolist(), color='r');

        #add days and plot
        for day in df_plt['date'].unique() : 
            plt.axvline(x=day, linewidth=1, color='b')
        plt.title('TagID: '+tag)
        #for label in ax.get_xticklabels():
        #    label.set_rotation(90) 
        y_ = np.nanmax(df_plt['Battery Voltage'].tolist())
        x_ = np.nanmin(df_plt['date'].unique())
        ax.text(x_, y_-0.1*(y_-np.nanmin(df_plt['Battery Voltage'].tolist())), '#device update in the last hour -right axis', fontsize=10, color='r')
        ax.text(x_, y_-0.2*(y_-np.nanmin(df_plt['Battery Voltage'].tolist())), '#transition in the last hour -right axis', fontsize=10, color='green')
        ax.text(x_, y_-0.3*(y_-np.nanmin(df_plt['Battery Voltage'].tolist())), 'Battery Voltage -left axis', fontsize=10, color='blue')

        #create a director if not existing
        path_incase = os.path.join(path_daily_ver, 'INcaseALLplot', SystemID.replace(':',''))
        if not os.path.exists(path_incase):
            os.makedirs(path_incase)
        plt.savefig(os.path.join(path_incase,id_run+'_'+str(tag)+'Tag_CHECK.png'), dpi=300,
                                format='png', bbox_inches='tight')
    except Exception as e:
                print('ERROR:-------------',e, '    TAG: ',tag)
    #plt.show()